In [1]:
# !nvidia-smi
!nvidia-smi --query-gpu=name --format=csv,noheader | wc -l

/usr/bin/sh: nvidia-smi: command not found
0


In [3]:
import os
import pandas as pd
import tqdm
import math

In [17]:
#Set the path to the data folder, datafile and output folder and files

op_folder_name = 'sept2022'

root_folder = '/users/kent/jmaharja/drugAbuse/'
output_folder = os.path.abspath(os.path.join(root_folder, 'output/'+ op_folder_name))
model_folder = os.path.abspath(os.path.join(output_folder, 'RoBERTaMLM/'))
tokenizer_folder = os.path.abspath(os.path.join(output_folder, 'TokRoBERTa/'))

datafile= '2020_01_01.csv'
testfile= '20161007.csv'
outputfile = 'submission.csv'

input_folder = os.path.abspath(os.path.join(root_folder, 'input/'))
datafile_path = os.path.abspath(os.path.join(input_folder, datafile))
testfile_path = os.path.abspath(os.path.join(input_folder, testfile))
outputfile_path = os.path.abspath(os.path.join(output_folder, outputfile))

In [15]:
train_df =pd.read_csv(datafile_path,lineterminator='\n',skipinitialspace=True, usecols= ['text'])
train_df.rename(columns={'text':'Tweet'}, inplace=True)
train_df = train_df.dropna()
train_df.shape

(1115630, 1)

# Build a Tokenizer

In [18]:
# Drop the files from the output dir
!mkdir -p {output_folder}
txt_files_dir = output_folder + "/text_split"

!rm -rf {txt_files_dir}
!mkdir {txt_files_dir}

In [19]:
# Store values in a dataframe column (Series object) to files, one file per record
# The prefix is a unique ID to avoid to overwrite a text file
def column_to_files(column, prefix, txt_files_dir):
    i=prefix
    for row in column.to_list():
      file_name = os.path.join(txt_files_dir, str(i)+'.txt')
      try:
        f = open(file_name, 'wb')
        f.write(row.encode('utf-8'))
        f.close()
      except Exception as e: 
        print(row, e) 
      i+=1
    return i

In [20]:
# data = train_df["Tweet"]
# data = data.replace("\n"," ")
# prefix = 0
# #Create a file for every description value
# prefix = column_to_files(data, prefix, txt_files_dir)
# print(prefix)

1115630


In [24]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
import torch
from torch.utils.data.dataset import Dataset

In [ ]:
%%time 
paths = [str(x) for x in Path('/users/kent/jmaharja/drugAbuse/output/sept2022/').glob("text_split/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=paths, vocab_size=8192, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])

In [29]:
#Save the Tokenizer to disk
tokenizer.save_model(tokenizer_folder)

['/users/kent/jmaharja/drugAbuse/output/TokRoBERTa/vocab.json',
 '/users/kent/jmaharja/drugAbuse/output/TokRoBERTa/merges.txt']

In [31]:
# Prepare the tokenizer
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [32]:
tokenizer.encode("cook some blue.")

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode("cook some blue.").special_tokens_mask

In [ ]:
tokenizer.encode("cook some blue.").tokens